# Combining datasets with Pandas

In this notebook, we will be learning more about combining dataframes in Pandas.

## Objectives

By the end of this notebook, you will be able 
- to combine dataframes using Pandas
- understand various joining methods that are available

Pandas has several methods such as ```pd.merge()```, ```df.join()```, ```df.concat()``` and ```df.merge()```. All of these do the same<br> things more or less, but there are small differences, which you will see in this session. he major differences between these, though,<br> largely depend on what they do by default when you try to combine different data. By default, ```merge()``` looks to join on common columns,<br> ```join()``` on common indices, and ```concat()``` by just appending on a given axis.



You can find more detail about the differences between all three of these in this [document](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html). We'll see some examples below which will give<br> you more clarity.

Let's use the same wine data for this purpose!

In [3]:
# We'll go back to our wine data set.
import pandas as pd
wine_df = pd.read_csv('data/winequality-red.csv', delimiter=';')
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
# A glance at the values of the quality of wine in the DataFrame
wine_df.quality.unique()

So, as you can see from above output, the wine quality is a catagorical variable, where the values goes from 3-8. In Data Science, we<br> use this method called, ```get_dummies``` to handle catagorical variables. This actually converts a catagorical variable to a dummy/indicator<br> variable, which is easy to use in the analysis afterwards.

In [4]:
# get_dummies is a method called on the pandas module - you simply pass in a pandas Series 
# or DataFrame, and it will convert a categorical variable into dummy/indicator variables. 
quality_dummies = pd.get_dummies(wine_df.quality, prefix='quality',dtype=int)
quality_dummies.head()

,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8
0,0,0,1,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,1,0,0
4,0,0,1,0,0,0


Now you can see how it did the magic right?? We have a new dataframe. To know more about ```get_dummies```, read the document [here](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).<br> We will be anyway deal with this method later in this course a lot!

Now let's look at the ```.join()``` method. Remember, this joins on indices by default. This means that we can simply join our quality<br> dummies dataframe back to our original wine dataframe with the following...

In [ ]:
joined_df = wine_df.join(quality_dummies)
joined_df.head() 

The arguments of ```.join``` are the following:

```DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)```

Now the question is **how** we can specify which join method we want to use?

Well, the table below shows you how and when to use them! It also contain the SQL equivalent ofeach of those commands too!

<img src="./images/join.png" width="400"/>

Let's have a look at the ```.concat()```option.

Its similar to join. We can specify the method we want to use combine the datasets.

Different from join and merge, which by default operate on columns, concat can define whether to operate on columns or rows. In the<br> images below, you can see the differences, if axis is set as 0 or 1.

Let's try few things using the data!

In [ ]:
joined_df2 = pd.concat([quality_dummies, wine_df], axis=1)
joined_df2.tail()

Let's read in a different data set, since we're looking at combining multiple data sources.

In [5]:
red_wines_df = pd.read_csv('data/winequality-red.csv', delimiter=';')
white_wines_df = pd.read_csv('data/winequality-white.csv', delimiter=';')

In [ ]:
# see the columns
red_wines_df.columns

In [ ]:
white_wines_df.columns

Let's do some groupby on these datasets based on the ```quality``` column and then get the mean value od ```fixed acidity``` for each<br> of the groups. And thereby create a new dataframe with those details:

In [7]:
red_wines_quality_df = red_wines_df.groupby('quality').mean()['fixed acidity'].reset_index()
red_wines_quality_df

,quality,fixed acidity
0,3,8.360000
1,4,7.779245
2,5,8.167254
3,6,8.347179
4,7,8.872362
5,8,8.566667


Do the same for white wines:

In [8]:
white_wines_quality_df = white_wines_df.groupby('quality').mean()['fixed acidity'].reset_index()
white_wines_quality_df

,quality,fixed acidity
0,3,7.600000
1,4,7.129448
2,5,6.933974
3,6,6.837671
4,7,6.734716
5,8,6.657143
6,9,7.420000


Now let's use 3 of the ```combining methods``` and see how it differs.

1. Use ```.merge``` on the above 2 dataframes on the column ```quality```.

In [9]:
pd.merge(red_wines_quality_df, white_wines_quality_df, on=['quality'], suffixes=[' red', ' white'])   #by default how ="inner"

,quality,fixed acidity red,fixed acidity white
0,3,8.360000,7.600000
1,4,7.779245,7.129448
2,5,8.167254,6.933974
3,6,8.347179,6.837671
4,7,8.872362,6.734716
5,8,8.566667,6.657143


2. Use ```.concat``` to combine

In [ ]:
pd.concat([red_wines_quality_df, white_wines_quality_df], axis=1,join='inner')  #by default join="outer"

# 🔍 Explanation:

# axis=0, join='inner'
# You're stacking two objects vertically (axis=0), so pandas is appending the rows.

# join='inner' says:

# "Only keep the columns that are common to both inputs."



# axis=1, join='inner'
# You are combining columns side by side (column-wise concat).

# join='inner' means:

# Only keep rows (index values) that are common to all DataFrames.


,quality,fixed acidity,quality,fixed acidity
0,3.0,8.360000,3,7.600000
1,4.0,7.779245,4,7.129448
2,5.0,8.167254,5,6.933974
3,6.0,8.347179,6,6.837671
4,7.0,8.872362,7,6.734716
5,8.0,8.566667,8,6.657143
6,NaN,NaN,9,7.420000


You can see that, you have to use an option called ```join``` along with ```.concat()```

3. Use ```.join``` to combine

In [ ]:
red_wines_quality_df.join(white_wines_quality_df, lsuffix='red', rsuffix='white') #by default how="left"

,qualityred,fixed acidityred,qualitywhite,fixed aciditywhite
0,3.0,8.360000,3,7.600000
1,4.0,7.779245,4,7.129448
2,5.0,8.167254,5,6.933974
3,6.0,8.347179,6,6.837671
4,7.0,8.872362,7,6.734716
5,8.0,8.566667,8,6.657143
6,NaN,NaN,9,7.420000


You can see that, these three options produced more or less the same results!

## Check your understanding!

Let's check the ideas that we have learnt:

1. Please join the two given dataframes (df1 and df2) along rows and merge with the third (df3) dataframe along the common column id.

In [61]:
df1 = pd.DataFrame({
        'student_id': ['1', '2', '3', '4', '5'],
         'name': ['Ellen', 'Noora', 'Hena', 'Dora', 'Ginny'], 
        'subject': ['Maths', 'Biology', 'Biology', 'English', 'Philosophy']})
df2 = pd.DataFrame({
        'student_id': ['6', '7', '8', '9', '10'],
        'name': ['Fransiska', 'Anna', 'Jena', 'Veronica', 'Paula'], 
        'subject': ['Chemistry', 'Economics', 'Maths', 'Maths', 'Social Science']})
df3 = pd.DataFrame({
        'student_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11', '12', '13'],
        'marks': [20, 15, 42, 27, 31, 25, 63, 44, 66, 73, 88, 72]})

In [62]:
df1

,student_id,name,subject
0,1,Ellen,Maths
1,2,Noora,Biology
2,3,Hena,Biology
3,4,Dora,English
4,5,Ginny,Philosophy


In [63]:
df2

,student_id,name,subject
0,6,Fransiska,Chemistry
1,7,Anna,Economics
2,8,Jena,Maths
3,9,Veronica,Maths
4,10,Paula,Social Science


In [64]:
df3

,student_id,marks
0,1,20
1,2,15
2,3,42
3,4,27
4,5,31
5,7,25
6,8,63
7,9,44
8,10,66
9,11,73


In [206]:

combined = pd.concat([df1, df2],axis=1)  #by default join="outer"
combined

,student_id,name,subject,student_id,name,subject
0,1,Ellen,Maths,6,Fransiska,Chemistry
1,2,Noora,Biology,7,Anna,Economics
2,3,Hena,Biology,8,Jena,Maths
3,4,Dora,English,9,Veronica,Maths
4,5,Ginny,Philosophy,10,Paula,Social Science


In [100]:
pd.merge(combined, df3, on=['student_id'])    #by default how ="inner"

,student_id,name,subject,marks
0,1,Ellen,Maths,20
1,2,Noora,Biology,15
2,3,Hena,Biology,42
3,4,Dora,English,27
4,5,Ginny,Philosophy,31
5,7,Anna,Economics,25
6,8,Jena,Maths,63
7,9,Veronica,Maths,44
8,10,Paula,Social Science,66


If you look at the resulting Dataframe, do you notice anything unusual? And if so, could there be any logical reason why it happened?

In [ ]:
#In resulting dataset got only 9 student_Ids because based on student_id we get only similar datas from both columns(6,11,12,13-got eliminated)

2. You have received some weather data (temperature) of the last year. For each month the average temperature was measured, only for a<br> few months the maximum temperature could be measured. Anyway, you want to combine these two data without losing any information.

(Extra question: Can you fill in the average max. Temperature for the missing values in the Column ```Max TemperatureF```)

In [158]:
weather_mean_data = {'Mean TemperatureF': [50.1, 60., 32.935, 28.714, 42.354, 72.870, 60.133, 35., 62.612, 39.8, 55.451, 63.766],
                     'Month': ['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep']}
weather_max_data = {'Max TemperatureF': [78, 89, 90, 81], 'Month': ['Jan', 'Apr', 'Jul', 'Oct']}



In [159]:

weather_mean_data = pd.DataFrame({'Mean TemperatureF': [50.1, 60., 32.935, 28.714, 42.354, 72.870, 60.133, 35., 62.612, 39.8, 55.451, 63.766],
                     'Month': ['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep']})

weather_mean_data

,Mean TemperatureF,Month
0,50.100,Apr
1,60.000,Aug
2,32.935,Dec
3,28.714,Feb
4,42.354,Jan
5,72.870,Jul
6,60.133,Jun
7,35.000,Mar
8,62.612,May
9,39.800,Nov


In [160]:
weather_max_data = pd.DataFrame({'Max TemperatureF': [78, 89, 90, 81], 'Month': ['Jan', 'Apr', 'Jul', 'Oct']})
weather_max_data

,Max TemperatureF,Month
0,78,Jan
1,89,Apr
2,90,Jul
3,81,Oct


In [184]:

weather_combined = weather_mean_data.merge(weather_max_data, on='Month',how='outer')
weather_combined

,Mean TemperatureF,Month,Max TemperatureF
0,50.100,Apr,89.0
1,60.000,Aug,NaN
2,32.935,Dec,NaN
3,28.714,Feb,NaN
4,42.354,Jan,78.0
5,72.870,Jul,90.0
6,60.133,Jun,NaN
7,35.000,Mar,NaN
8,62.612,May,NaN
9,39.800,Nov,NaN


In [1]:

weather_combined['Max TemperatureF'].fillna(weather_combined['Max TemperatureF'].mean(),inplace=True)
weather_combined

NameError: name 'weather_combined' is not defined